# LSTM For Predictive Maintenance

### Loading Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline  

### Loading Dataset

In [3]:
dataset_train=pd.read_csv('https://raw.githubusercontent.com/sudhanvabayar/LSTM/master/data/PM_train.txt',sep=' ',header=None).drop([26,27],axis=1)
col_names = ['id','cycle','setting1','setting2','setting3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']
dataset_train.columns=col_names
print('Shape of Train dataset: ',dataset_train.shape)
dataset_train.head()

Shape of Train dataset:  (20631, 26)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [4]:
dataset_test=pd.read_csv('https://raw.githubusercontent.com/sudhanvabayar/LSTM/master/data/PM_test.txt',sep=' ',header=None).drop([26,27],axis=1)
dataset_test.columns=col_names
#dataset_test.head()
print('Shape of Test dataset: ',dataset_train.shape)
dataset_train.head()

Shape of Test dataset:  (20631, 26)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [36]:
dataset_test.shape

(13096, 27)

#### Loadind Truth table

In [5]:
pm_truth=pd.read_csv('https://raw.githubusercontent.com/sudhanvabayar/LSTM/master/data/PM_truth.txt',sep=' ',header=None).drop([1],axis=1)
pm_truth.columns=['more']
pm_truth['id']=pm_truth.index+1
pm_truth.head()

,more,id
0,112,1
1,98,2
2,69,3
3,82,4
4,91,5


In [6]:
# generate column max for test data
rul = pd.DataFrame(dataset_test.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
rul.head()

,id,max
0,1,31
1,2,49
2,3,126
3,4,106
4,5,98


In [7]:
# run to failure
pm_truth['rtf']=pm_truth['more']+rul['max']
pm_truth.head()

,more,id,rtf
0,112,1,143
1,98,2,147
2,69,3,195
3,82,4,188
4,91,5,189


In [8]:
pm_truth.drop('more', axis=1, inplace=True)
dataset_test=dataset_test.merge(pm_truth,on=['id'],how='left')
dataset_test['ttf']=dataset_test['rtf'] - dataset_test['cycle']
dataset_test.drop('rtf', axis=1, inplace=True)
dataset_test.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,ttf
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,142
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,141
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,140
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,139
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,138


In [9]:
dataset_train['ttf'] = dataset_train.groupby(['id'])['cycle'].transform(max)-dataset_train['cycle']
dataset_train.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,ttf
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [10]:
df_train=dataset_train.copy()
df_test=dataset_test.copy()
period=30
df_train['label_bc'] = df_train['ttf'].apply(lambda x: 1 if x <= period else 0)
df_test['label_bc'] = df_test['ttf'].apply(lambda x: 1 if x <= period else 0)
df_train.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,ttf,label_bc
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0


In [11]:
features_col_name=['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11',
                   's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
target_col_name='label_bc'

## Feature Scaling

In [12]:
sc=MinMaxScaler()
df_train[features_col_name]=sc.fit_transform(df_train[features_col_name])
df_test[features_col_name]=sc.transform(df_test[features_col_name])

In [13]:
df_train.shape[1]

28

In [14]:
df_train.columns

Index(['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
       's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
       's15', 's16', 's17', 's18', 's19', 's20', 's21', 'ttf', 'label_bc'],
      dtype='object')

In [15]:
df_test.columns

Index(['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
       's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
       's15', 's16', 's17', 's18', 's19', 's20', 's21', 'ttf', 'label_bc'],
      dtype='object')

In [16]:
train_target = df_train['label_bc']
test_target = df_test['label_bc']
train_id = df_train['id']
test_id = df_test['id']

df_train.drop(['label_bc'], axis=1, inplace=True)
df_train.drop(['id'], axis=1, inplace=True)
df_test.drop(['label_bc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

In [17]:
df_train.columns

Index(['cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4',
       's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15',
       's16', 's17', 's18', 's19', 's20', 's21', 'ttf'],
      dtype='object')

In [18]:
# define the number of features
ncol = df_train.shape[1]

In [19]:
from sklearn.model_selection import train_test_split
from numpy.random import seed

In [21]:
X1_train, X1_test, Y1_train, Y1_test = train_test_split(df_train, train_target, train_size = 0.9, random_state = seed(2017))

In [22]:
### Define the encoder dimension
encoding_dim = 10

In [23]:
from keras.layers import Input, Dense

In [24]:
input_dim = Input(shape = (ncol, ))

# Encoder Layers
encoded1 = Dense(1000, activation = 'relu')(input_dim)
encoded2 = Dense(500, activation = 'relu')(encoded1)
encoded3 = Dense(250, activation = 'relu')(encoded2)
encoded4 = Dense(encoding_dim, activation = 'relu')(encoded3)

# Decoder Layers
decoded1 = Dense(250, activation = 'relu')(encoded4)
decoded2 = Dense(500, activation = 'relu')(decoded1)
decoded3 = Dense(1000, activation = 'relu')(decoded2)
decoded4 = Dense(ncol, activation = 'sigmoid')(decoded3)

W0806 09:53:04.196038 140457244559168 deprecation_wrapper.py:119] From /srv/conda/envs/notebook/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 09:53:04.215046 140457244559168 deprecation_wrapper.py:119] From /srv/conda/envs/notebook/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 09:53:04.224060 140457244559168 deprecation_wrapper.py:119] From /srv/conda/envs/notebook/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [25]:
# Combine Encoder and Deocder layers
from keras.models import Model

autoencoder = Model(inputs = input_dim, outputs = decoded4)

In [26]:
# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')

W0806 09:54:53.602547 140457244559168 deprecation_wrapper.py:119] From /srv/conda/envs/notebook/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0806 09:54:53.624729 140457244559168 deprecation_wrapper.py:119] From /srv/conda/envs/notebook/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0806 09:54:53.629747 140457244559168 deprecation.py:323] From /srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [27]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              27000     
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_3 (Dense)              (None, 250)               125250    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2510      
_________________________________________________________________
dense_5 (Dense)              (None, 250)               2750      
_________________________________________________________________
dense_6 (Dense)              (None, 500)               125500    
__________

In [28]:
autoencoder.fit(X1_train, X1_train, nb_epoch = 10, batch_size = 32, shuffle = False, validation_data = (X1_test, X1_test))

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
W0806 09:55:20.841285 140457244559168 deprecation_wrapper.py:119] From /srv/conda/envs/notebook/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 18567 samples, validate on 2064 samples
Epoch 1/10
18567/18567 [==============================] - 55s 3ms/step - loss: -130.3752 - val_loss: -131.1359
Epoch 2/10
18567/18567 [==============================] - 49s 3ms/step - loss: -131.1948 - val_loss: -131.1349
Epoch 3/10
18567/18567 [==============================] - 52s 3ms/step - loss: -131.2035 - val_loss: -131.1363
Epoch 4/10
18567/18567 [==============================] - 57s 3ms/step - loss: -131.2011 - val_loss: -131.1365
Epoch 5/10
18567/18567 [==============================] - 50s 3ms/step - loss: -130.8695 - val_loss: -131.1270
Epoch 6/10
18567/18567 [==============================] - 57s 3ms/step - loss: -131.2032 - val_loss: -131.1367
Epoch 7/10
18567/18567 [==============================] - 58s 3ms/step - loss: -131.2047 - val_loss: -131.1369
Epoch 8/10
18567/18567 [==============================] - 59s 3ms/step - loss: -131.2037 - val_loss: -131.1367
Epoch 9/10
18567/18567 [==============================] - 58s 3

In [29]:
encoder = Model(inputs = input_dim, outputs = encoded4)
encoded_input = Input(shape = (encoding_dim, ))

In [30]:
encoded_train = pd.DataFrame(encoder.predict(df_train))
encoded_train = encoded_train.add_prefix('feature_')

encoded_test = pd.DataFrame(encoder.predict(df_test))
encoded_test = encoded_test.add_prefix('feature_')

In [31]:
encoded_train['label_bc'] = train_target
encoded_train['id'] = train_id

In [32]:
encoded_test['label_bc'] = test_target
encoded_test['id'] = test_id

In [33]:
print(encoded_train.shape)
encoded_train.head()

(20631, 12)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,label_bc,id
0,0.0,0.0,67.999092,0.0,0.0,0.0,0.0,0.0,323.556885,0.0,0,1
1,0.0,0.0,71.171783,0.0,0.0,0.0,0.0,0.0,324.361664,0.0,0,1
2,0.0,0.0,73.057793,0.0,0.0,0.0,0.0,0.0,324.453918,0.0,0,1
3,0.0,0.0,75.505699,0.0,0.0,0.0,0.0,0.0,324.960724,0.0,0,1
4,0.0,0.0,76.783119,0.0,0.0,0.0,0.0,0.0,324.019592,0.0,0,1


In [34]:
print(encoded_test.shape)
encoded_test.head()

(13096, 12)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,label_bc,id
0,0.0,0.0,59.335281,0.0,0.0,0.0,0.0,0.0,247.975998,0.0,0,1
1,0.0,0.0,60.550934,0.0,0.0,0.0,0.0,0.0,247.237183,0.0,0,1
2,0.0,0.0,64.644112,0.0,0.0,0.0,0.0,0.0,248.860306,0.0,0,1
3,0.0,0.0,65.202576,0.0,0.0,0.0,0.0,0.0,247.735916,0.0,0,1
4,0.0,0.0,67.394821,0.0,0.0,0.0,0.0,0.0,248.123199,0.0,0,1


In [37]:
encoded_train.columns

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'label_bc', 'id'],
      dtype='object')

In [41]:
features_col_name_1 = ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9']

## Function to reshape dataset as required by LSTM

In [42]:
def gen_sequence(id_df, seq_length, seq_cols):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        lstm_array.append(data_array[start:stop, :])
    return np.array(lstm_array)

# function to generate labels
def gen_label(id_df, seq_length, seq_cols,label):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    y_label=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        y_label.append(id_df[label][stop])
    return np.array(y_label)

In [43]:
# timestamp or window size
seq_length=50
seq_cols=features_col_name_1

In [44]:
# generate X_train
X_train=np.concatenate(list(list(gen_sequence(encoded_train[encoded_train['id']==id], seq_length, seq_cols)) for id in encoded_train['id'].unique()))
print(X_train.shape)

(20531, 50, 10)


In [45]:
# generate y_train
y_train=np.concatenate(list(list(gen_label(encoded_train[encoded_train['id']==id], 50, seq_cols,'label_bc')) for id in encoded_train['id'].unique()))
print(y_train.shape)

(20531,)


In [46]:
# generate X_test
X_test=np.concatenate(list(list(gen_sequence(encoded_test[encoded_test['id']==id], seq_length, seq_cols)) for id in encoded_test['id'].unique()))
print(X_test.shape)
# generate y_test
y_test=np.concatenate(list(list(gen_label(encoded_test[encoded_test['id']==id], 50, seq_cols,'label_bc')) for id in encoded_test['id'].unique()))
print(y_test.shape)

(12996, 50, 10)
(12996,)


## LSTM Network

In [47]:
nb_features =X_train.shape[2]
timestamp=seq_length

model = Sequential()

model.add(LSTM(
         input_shape=(timestamp, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

W0806 10:16:07.881103 140457244559168 deprecation.py:506] From /srv/conda/envs/notebook/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 100)           44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 51        
Total params: 74,651
Trainable params: 74,651
Non-trainable params: 0
_________________________________________________________________


In [48]:
# fit the network
model.fit(X_train, y_train, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

Train on 19504 samples, validate on 1027 samples
Epoch 1/10
19504/19504 [==============================] - 100s 5ms/step - loss: 0.0913 - acc: 0.9742 - val_loss: 0.0454 - val_acc: 0.9825
Epoch 2/10
19504/19504 [==============================] - 91s 5ms/step - loss: 0.0347 - acc: 0.9868 - val_loss: 0.0193 - val_acc: 0.9922
Epoch 3/10
19504/19504 [==============================] - 92s 5ms/step - loss: 0.0252 - acc: 0.9903 - val_loss: 0.0432 - val_acc: 0.9786


In [49]:
# training metrics
scores = model.evaluate(X_train, y_train, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

20531/20531 [==============================] - 24s 1ms/step
Accurracy: 0.981832366050419


In [50]:
y_pred=model.predict_classes(X_test)
print('Accuracy of model on test data: ',accuracy_score(y_test,y_pred))
print('Confusion Matrix: \n',confusion_matrix(y_test,y_pred))

Accuracy of model on test data:  0.992459218220991
Confusion Matrix: 
 [[12661     3]
 [   95   237]]


### Probability of Machine failure

In [53]:
def prob_failure(machine_id):
    machine_df=encoded_test[encoded_test.id==machine_id]
    machine_test=gen_sequence(machine_df,seq_length,seq_cols)
    m_pred=model.predict(machine_test)
    failure_prob=list(m_pred[-1]*100)[0]
    return failure_prob

In [61]:
machine_id=16
print('Probability that machine will fail within 30 days: ',prob_failure(machine_id))

Probability that machine will fail within 30 days:  0.048023462
